In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers, optimizers
from keras.models import Model
from keras import layers
from keras import models
from keras.layers import BatchNormalization
from tensorflow.keras.utils import Sequence

import os, shutil
import numpy as np
import time
import io
from PIL import Image


from keras import metrics
import functools
from functools import partial

import matplotlib.pyplot as plt

In [ ]:
# Load the VGG16 model
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(72, 96, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 72, 96, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 72, 96, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 72, 96, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 36, 48, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 36, 48, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 36, 48, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 18, 24, 128)       0     

In [ ]:
inp = conv_base.input
out =conv_base.layers[-1].output

In [ ]:
thermalModel = Model(inp, out)

In [ ]:
#From the first 10 layers, only the last three are set up to allow the training.
cont = 0
for layer in thermalModel.layers:
    cont = cont + 1
    if (cont >= 8):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
thermalModel.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 72, 96, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 72, 96, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 72, 96, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 36, 48, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 36, 48, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 36, 48, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 18, 24, 128)       0     

In [ ]:
def count_directories(path):
    return len([name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))])

train_classes = count_directories('/content/drive/MyDrive/ExtractedTerravicDatabase_subset/train')
test_classes = count_directories('/content/drive/MyDrive/ExtractedTerravicDatabase_subset/test')
validation_classes = count_directories('/content/drive/MyDrive/ExtractedTerravicDatabase_subset/val')

print(f"Training classes: {train_classes}")
print(f"Testing classes: {test_classes}")
print(f"Validation classes: {validation_classes}")

Training classes: 16
Testing classes: 16
Validation classes: 16


In [ ]:
# Directories for train, val, test
train_dir = '/content/drive/MyDrive/ExtractedTerravicDatabase_subset/train'
val_dir = '/content/drive/MyDrive/ExtractedTerravicDatabase_subset/val'
test_dir = '/content/drive/MyDrive/ExtractedTerravicDatabase_subset/test'


In [ ]:
model = models.Sequential()
model.add(thermalModel)
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 2, 3, 512)         14714688  
                                                                 
 max_pooling2d (MaxPooling2  (None, 1, 1, 512)         0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 1, 1, 512)         2048      
 Normalization)                                                  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 16)                8208      
                                                                 
Total params: 14724944 (56.17 MB)
Trainable params: 1446

In [ ]:
top2_acc = functools.partial(metrics.top_k_categorical_accuracy, k=2)
top3_acc = functools.partial(metrics.top_k_categorical_accuracy, k=3)
top4_acc = functools.partial(metrics.top_k_categorical_accuracy, k=4)
top5_acc = functools.partial(metrics.top_k_categorical_accuracy, k=5)
top2_acc.__name__ = 'top2_acc'
top3_acc.__name__ = 'top3_acc'
top4_acc.__name__ = 'top4_acc'
top5_acc.__name__ = 'top5_acc'

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4), #Decrease learning rate
              metrics=['accuracy',top2_acc, top3_acc, top4_acc, top5_acc])

In [ ]:
# Image size and batch size
image_size = (72, 96)
batch_size = 4

In [ ]:
# Only rescaling for training, validation, and test data
datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical'
)


Found 2136 images belonging to 16 classes.
Found 1137 images belonging to 16 classes.
Found 1139 images belonging to 16 classes.


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    workers=0,
    max_queue_size=0
)


Epoch 1/10
534/534 [==============================] - 768s 1s/step - loss: 0.8537 - accuracy: 0.7790 - top2_acc: 0.8591 - top3_acc: 0.8919 - top4_acc: 0.9167 - top5_acc: 0.9321 - val_loss: 0.6864 - val_accuracy: 0.8125 - val_top2_acc: 0.8653 - val_top3_acc: 0.9252 - val_top4_acc: 0.9569 - val_top5_acc: 0.9798
Epoch 2/10
534/534 [==============================] - 18s 33ms/step - loss: 0.0626 - accuracy: 0.9911 - top2_acc: 0.9981 - top3_acc: 0.9986 - top4_acc: 0.9991 - top5_acc: 0.9995 - val_loss: 0.9016 - val_accuracy: 0.8415 - val_top2_acc: 0.8741 - val_top3_acc: 0.9085 - val_top4_acc: 0.9261 - val_top5_acc: 0.9296
Epoch 3/10
534/534 [==============================] - 19s 36ms/step - loss: 0.0277 - accuracy: 0.9958 - top2_acc: 0.9995 - top3_acc: 0.9995 - top4_acc: 0.9995 - top5_acc: 0.9995 - val_loss: 0.1371 - val_accuracy: 0.9525 - val_top2_acc: 0.9886 - val_top3_acc: 0.9991 - val_top4_acc: 1.0000 - val_top5_acc: 1.0000
Epoch 4/10
534/534 [==============================] - 18s 34ms/st

In [ ]:
# evaluation_results = model.evaluate(test_generator, steps=test_generator.num_samples // batch_size)

# # Unpack the evaluation results
# test_loss, test_acc, top2_acc, top3_acc, top4_acc, top5_acc = evaluation_results

# print(f'Test loss: {test_loss}')
# print(f'Test accuracy: {test_acc}')
# print(f'Top-2 accuracy: {top2_acc}')
# print(f'Top-3 accuracy: {top3_acc}')
# print(f'Top-4 accuracy: {top4_acc}')
# print(f'Top-5 accuracy: {top5_acc}')

In [ ]:
model = models.Sequential()
model.add(thermalModel)
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='softmax'))

In [ ]:
# Verify the output of the data generator
batch_x, batch_y = next(iter(train_generator))

print(f'Batch X shape: {batch_x.shape}')
print(f'Batch Y shape: {batch_y.shape}')

Batch X shape: (4, 72, 96, 3)
Batch Y shape: (4, 16)


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy',top2_acc, top3_acc, top4_acc, top5_acc],
              run_eagerly=True)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=3,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    workers=0,
    max_queue_size=0
)

Epoch 1/3


534/534 [==============================] - 61s 109ms/step - loss: 0.1548 - accuracy: 0.9757 - top2_acc: 0.9831 - top3_acc: 0.9874 - top4_acc: 0.9897 - top5_acc: 0.9925 - val_loss: 5.7095e-04 - val_accuracy: 1.0000 - val_top2_acc: 1.0000 - val_top3_acc: 1.0000 - val_top4_acc: 1.0000 - val_top5_acc: 1.0000
Epoch 2/3
534/534 [==============================] - 57s 106ms/step - loss: 0.0094 - accuracy: 0.9986 - top2_acc: 0.9995 - top3_acc: 0.9995 - top4_acc: 0.9995 - top5_acc: 1.0000 - val_loss: 4.1984e-05 - val_accuracy: 1.0000 - val_top2_acc: 1.0000 - val_top3_acc: 1.0000 - val_top4_acc: 1.0000 - val_top5_acc: 1.0000
Epoch 3/3
534/534 [==============================] - 58s 108ms/step - loss: 0.0069 - accuracy: 0.9986 - top2_acc: 1.0000 - top3_acc: 1.0000 - top4_acc: 1.0000 - top5_acc: 1.0000 - val_loss: 9.3158e-06 - val_accuracy: 1.0000 - val_top2_acc: 1.0000 - val_top3_acc: 1.0000 - val_top4_acc: 1.0000 - val_top5_acc: 1.0000


In [ ]:
#Test stage
test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=(72, 96),
        batch_size=74,
        color_mode='rgb',
        class_mode='categorical')
start_time = time.time()
evaluation_results = model.evaluate(test_generator, steps=16, workers=0, max_queue_size=0)
# test_loss, test_acc, test_top2_acc, test_top3_acc = model.evaluate_generator(test_generator, steps=16, workers=0, max_queue_size=0)
end_time = time.time()
testing_time = end_time - start_time

# Unpack and print evaluation results
test_loss, test_acc, test_top2_acc, test_top3_acc, test_top4_acc, test_top5_acc = evaluation_results

print('test acc:', test_acc)
print('Testing time:', testing_time)
print('Recognition rate top2: ', test_top2_acc)
print('Recognition rate top3: ', test_top3_acc)
print('Recognition rate top3: ', test_top4_acc)
print('Recognition rate top3: ', test_top5_acc)

Found 1139 images belonging to 16 classes.
16/16 [==============================] - 258s 17s/step - loss: 9.3678e-06 - accuracy: 1.0000 - top2_acc: 1.0000 - top3_acc: 1.0000 - top4_acc: 1.0000 - top5_acc: 1.0000
test acc: 1.0
Testing time: 281.04308128356934
Recognition rate top2:  1.0
Recognition rate top3:  1.0
Recognition rate top3:  1.0
Recognition rate top3:  1.0


In [ ]:
#Processing time
testing_time / 1184

0.23736746730031194

In [ ]:
model.save('thermal_face_recognition_vgg16.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#Test the model with the TV dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# List directories in /content
directories = [d for d in os.listdir('/content') if os.path.isdir(os.path.join('/content', d))]
print(directories)

['.config', 'drive', 'sample_data']


In [ ]:
shared_dir = '/content/drive/MyDrive/normalized/validate/thermal'

In [ ]:
# List the contents of the shared drive
os.listdir(shared_dir)

['class_4',
 'class_6',
 'class_9',
 'class_12',
 'class_2',
 'class_7',
 'class_13',
 'class_16',
 'class_10',
 'class_11',
 'class_8',
 'class_3',
 'class_14',
 'class_5',
 'class_15',
 'class_1']

In [ ]:
#Path where my dataset is stored
base_dir2 = '/content/drive/MyDrive/normalized/validate/thermal'
base_dir3 = '/content/drive/MyDrive/normalized/test/thermal'
base_dir4 = '/content/drive/MyDrive/normalized/train/thermal'
# Directories for test splits
test_dir1 = os.path.join(base_dir2)
test_dir2 = os.path.join(base_dir3)
test_dir3 = os.path.join(base_dir4)

In [ ]:
os.listdir(base_dir2)

['class_4',
 'class_6',
 'class_9',
 'class_12',
 'class_2',
 'class_7',
 'class_13',
 'class_16',
 'class_10',
 'class_11',
 'class_8',
 'class_3',
 'class_14',
 'class_5',
 'class_15',
 'class_1']

In [ ]:
os.listdir(base_dir3)

['class_10',
 'class_14',
 'class_1',
 'class_13',
 'class_3',
 'class_16',
 'class_7',
 'class_11',
 'class_5',
 'class_6',
 'class_12',
 'class_4',
 'class_2',
 'class_15',
 'class_8',
 'class_9']

In [ ]:
test_datagen2 = ImageDataGenerator(rescale=1./255)

test_generator2 = test_datagen2.flow_from_directory(
    test_dir1,
    target_size=(72, 96),
    batch_size=148,
    color_mode='rgb',
    class_mode='categorical')

Found 79 images belonging to 16 classes.


In [ ]:
# test_datagen3 = ImageDataGenerator(rescale=1./255)

# test_generator3 = test_datagen3.flow_from_directory(
#     test_dir2,
#     target_size=(72, 96),
#     batch_size=148,
#     color_mode='rgb',
#     class_mode='categorical')

In [ ]:
test_datagen4 = ImageDataGenerator(rescale=1./255)

test_generator4 = test_datagen4.flow_from_directory(
    test_dir3,
    target_size=(72, 96),
    batch_size=148,
    color_mode='rgb',
    class_mode='categorical')

Found 72 images belonging to 16 classes.


In [ ]:
test_loss, test_acc, test_top2_acc, test_top3_acc, test_top4_acc, test_top5_acc = model.evaluate(test_generator2, steps=16, workers=0, max_queue_size=0)
print('Recognition rate top1: ', test_acc)
print('Recognition rate top2: ', test_top2_acc)
print('Recognition rate top3: ', test_top3_acc)
print('Recognition rate top4: ', test_top4_acc)
print('Recognition rate top5: ', test_top5_acc)

 1/16 [>.............................] - ETA: 52s - loss: 7.0761 - accuracy: 0.0127 - top2_acc: 0.0886 - top3_acc: 0.1392 - top4_acc: 0.2405 - top5_acc: 0.2911

16/16 [==============================] - 3s 616us/step - loss: 7.0761 - accuracy: 0.0127 - top2_acc: 0.0886 - top3_acc: 0.1392 - top4_acc: 0.2405 - top5_acc: 0.2911
Recognition rate top1:  0.012658228166401386
Recognition rate top2:  0.08860759437084198
Recognition rate top3:  0.13924050331115723
Recognition rate top4:  0.2405063360929489
Recognition rate top5:  0.29113924503326416


In [ ]:
# test_loss, test_acc, test_top2_acc, test_top3_acc, test_top4_acc, test_top5_acc = model.evaluate(test_generator3, steps=16, workers=0, max_queue_size=0)
# print('Recognition rate top1: ', test_acc)
# print('Recognition rate top2: ', test_top2_acc)
# print('Recognition rate top3: ', test_top3_acc)
# print('Recognition rate top4: ', test_top4_acc)
# print('Recognition rate top5: ', test_top5_acc)

In [ ]:
test_loss, test_acc, test_top2_acc, test_top3_acc, test_top4_acc, test_top5_acc = model.evaluate(test_generator4, steps=16, workers=0, max_queue_size=0)
print('Recognition rate top1: ', test_acc)
print('Recognition rate top2: ', test_top2_acc)
print('Recognition rate top3: ', test_top3_acc)
print('Recognition rate top4: ', test_top4_acc)
print('Recognition rate top5: ', test_top5_acc)

 1/16 [>.............................] - ETA: 48s - loss: 6.7227 - accuracy: 0.0694 - top2_acc: 0.1944 - top3_acc: 0.3750 - top4_acc: 0.3889 - top5_acc: 0.4028

16/16 [==============================] - 3s 588us/step - loss: 6.7227 - accuracy: 0.0694 - top2_acc: 0.1944 - top3_acc: 0.3750 - top4_acc: 0.3889 - top5_acc: 0.4028
Recognition rate top1:  0.0694444477558136
Recognition rate top2:  0.1944444477558136
Recognition rate top3:  0.375
Recognition rate top4:  0.3888888955116272
Recognition rate top5:  0.4027777910232544


# model improvement

In [ ]:
# Define paths to your data directories
dir_val = '/content/drive/MyDrive/normalized/validate/thermal'
dir_test = '/content/drive/MyDrive/normalized/test/thermal'
dir_train = '/content/drive/MyDrive/normalized/train/thermal'

In [ ]:
# Create data generators with data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_new_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(72, 96),
    batch_size=batch_size,
    class_mode='categorical'
)

val_new_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(72, 96),
    batch_size=batch_size,
    class_mode='categorical'
)

test_new_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(72, 96),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2136 images belonging to 16 classes.
Found 1137 images belonging to 16 classes.
Found 1139 images belonging to 16 classes.


In [ ]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Load the saved model
othermodel = load_model('/content/drive/MyDrive/thermal_face_recognition_vgg16.h5', custom_objects={'top2_acc': top2_acc, 'top3_acc': top3_acc, 'top4_acc': top4_acc, 'top5_acc': top5_acc})

In [ ]:
# Unfreeze the top layers of the model
for layer in othermodel.layers[-10:]:
    layer.trainable = True

In [ ]:
othermodel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 2, 3, 512)         14714688  
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 1, 1, 512)         0         
 g2D)                                                            
                                                                 
 batch_normalization_1 (Bat  (None, 1, 1, 512)         2048      
 chNormalization)                                                
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 16)                8208      
                                                                 
Total params: 14724944 (56.17 MB)
Trainable params: 14

In [ ]:
# Compile the model with a lower learning rate
othermodel.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy', top2_acc, top3_acc, top4_acc, top5_acc])
# Fine-tune the model
history = othermodel.fit(
    train_new_generator,
    steps_per_epoch=len(train_new_generator),
    epochs=3,
    validation_data=val_new_generator,
    validation_steps=len(val_new_generator)
)

Epoch 1/3
534/534 [==============================] - 28s 45ms/step - loss: 0.3854 - accuracy: 0.8876 - top2_acc: 0.9457 - top3_acc: 0.9625 - top4_acc: 0.9733 - top5_acc: 0.9775 - val_loss: 4.1923e-05 - val_accuracy: 1.0000 - val_top2_acc: 1.0000 - val_top3_acc: 1.0000 - val_top4_acc: 1.0000 - val_top5_acc: 1.0000
Epoch 2/3
534/534 [==============================] - 23s 43ms/step - loss: 0.2346 - accuracy: 0.9443 - top2_acc: 0.9696 - top3_acc: 0.9780 - top4_acc: 0.9836 - top5_acc: 0.9864 - val_loss: 6.4183e-06 - val_accuracy: 1.0000 - val_top2_acc: 1.0000 - val_top3_acc: 1.0000 - val_top4_acc: 1.0000 - val_top5_acc: 1.0000
Epoch 3/3
534/534 [==============================] - 24s 45ms/step - loss: 0.2068 - accuracy: 0.9396 - top2_acc: 0.9719 - top3_acc: 0.9813 - top4_acc: 0.9878 - top5_acc: 0.9911 - val_loss: 1.5676e-04 - val_accuracy: 1.0000 - val_top2_acc: 1.0000 - val_top3_acc: 1.0000 - val_top4_acc: 1.0000 - val_top5_acc: 1.0000


In [ ]:
# Evaluate the model on the test set
evaluation_results = othermodel.evaluate(test_generator, steps=len(test_generator))
print(evaluation_results)

# Unpack and print evaluation results
test_loss, test_acc, test_top2_acc, test_top3_acc, test_top4_acc, test_top5_acc = evaluation_results

print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_acc}')
print(f'Top-2 accuracy: {test_top2_acc}')
print(f'Top-3 accuracy: {test_top3_acc}')
print(f'Top-4 accuracy: {test_top4_acc}')
print(f'Top-5 accuracy: {test_top5_acc}')

285/285 [==============================] - 9s 32ms/step - loss: 1.8197e-04 - accuracy: 1.0000 - top2_acc: 1.0000 - top3_acc: 1.0000 - top4_acc: 1.0000 - top5_acc: 1.0000
[0.00018197059398517013, 1.0, 1.0, 1.0, 1.0, 1.0]
Test loss: 0.00018197059398517013
Test accuracy: 1.0
Top-2 accuracy: 1.0
Top-3 accuracy: 1.0
Top-4 accuracy: 1.0
Top-5 accuracy: 1.0


In [ ]:
# Save the model
othermodel.save('path_to_save_model/finetuned_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


more improvements:

- Cross-Validation: Use cross-validation to check model consistency across different data splits.

- Confusion Matrix: Generate a confusion matrix to identify specific misclassifications.

- Regularization and Error Analysis: Use regularization techniques and manually inspect errors to understand and mitigate model shortcomings.